# Oxford5k + MPEG7 Image Retrieval:

Clean version of all functions used to compute the metrics/Image retrieval of either MPEG7 or Oxford5k dataset

At the bottom of the page is the functions to run them, and instructions on inputted variables

## Imports

In [7]:
import numpy as np 
#import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import clear_output
import time
import pandas
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

## Loading in Oxford Test data

In [4]:
def load_test_data(images_path, gt_path):
    test_images_path = []
    test_names = []
    test_gray_images = []
    test_colour_images = []
    
    for filename in sorted(os.listdir(gt_path)):
        if filename.endswith("query.txt"):
            
            # Saving filename
            tmp = filename.split(".")[0].split("_")
            if len(tmp) == 4:
                name = tmp[0]+"_"+tmp[1]
            elif len(tmp) == 3:
                name = tmp[0]
            test_names.append(name)

            # Reading the image number to be saved
            with open(os.path.join(gt_path, filename), "r") as f:
                line = f.readline()
                test_images_path.append(line.split(" ")[0])
    
    for path in test_images_path:
        image = cv2.imread(os.path.join(images_path, path[5:]) + ".jpg")
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        test_gray_images.append(gray_image)
        test_colour_images.append(image)
    
    print("Loaded in {} Images!".format(len(test_gray_images)))
    
    return test_gray_images, test_colour_images, test_names, test_images_path

## Loading in Oxford Training data

In [8]:
BASE_DIR = "/home/sean/Code/Pawsey/oxford_data/"

def load_data(name):
    """Returns a dictionary of attributes and 1 of features for train or test data"""

    attributes = {}
    
    for img_att in ["names", "pixels", "images"]:
        attributes[img_att] = np.load(BASE_DIR + name + "_" + img_att + ".npy", allow_pickle=True)

    features = {}

    os.chdir(BASE_DIR + "NPY files for BoVW/")
    for bovw_size in os.listdir():
        features[bovw_size] = pandas.DataFrame(np.load(bovw_size + "/BoW_" + name.capitalize() + ".npy"))
    
    return(attributes, features)

(test_attributes, test_features) = load_data("test")
(train_attributes, train_features) = load_data("train")
train_features["bovw files for 10 Words"]

,0,1,2,3,4,5,6,7,8,9
0,0.314357,0.166189,0.288328,0.203231,0.247281,0.408464,0.269306,0.321365,0.408464,0.422480
1,0.323421,0.171378,0.285630,0.188076,0.267174,0.565107,0.249596,0.313753,0.304965,0.323421
2,0.382649,0.202579,0.280109,0.296366,0.217585,0.437671,0.257600,0.335131,0.290113,0.380148
3,0.298072,0.195457,0.276083,0.195457,0.305401,0.559495,0.309066,0.285856,0.273640,0.316396
4,0.336860,0.148814,0.304391,0.257042,0.228632,0.274629,0.247572,0.403149,0.347683,0.482968
...,...,...,...,...,...,...,...,...,...,...
562,0.321308,0.251836,0.334334,0.238810,0.258349,0.492817,0.240981,0.329992,0.240981,0.360386
563,0.312773,0.237406,0.221256,0.224486,0.179266,0.216949,0.197031,0.332153,0.299315,0.654617
564,0.352977,0.186982,0.301461,0.268072,0.233728,0.417848,0.262348,0.374919,0.237544,0.427388
565,0.222150,0.391848,0.283858,0.212894,0.235520,0.376421,0.280773,0.266374,0.401104,0.404190


In [15]:
import os
import numpy as np

# setting path name (make sure this notebook and the oxford_data folder are in the same directory/folder)
os.chdir("/home/sean/Code/Pawsey/")
path = os.getcwd()
path = os.path.join(path, "oxford_data")

# Loading in Images:
test_images = np.load(os.path.join(path, "test_images.npy"), allow_pickle=True)
train_images = np.load(os.path.join(path, "train_images.npy"), allow_pickle=True)
print("Loaded in {} Test images | {} Train images\n".format(len(test_images), len(train_images)))

# Loading in Names:
test_names = np.load(os.path.join(path, "test_names.npy"), allow_pickle=True)
train_names= np.load(os.path.join(path, "train_names.npy"), allow_pickle=True)
print("Loaded in {} Test names | {} Train names\n".format(len(test_names), len(train_names)))

# Loading in Pixel features:pixels
test_pixels = np.load(os.path.join(path, "test_pixels.npy"), allow_pickle=True)
train_pixels= np.load(os.path.join(path, "train_pixels.npy"), allow_pickle=True)
print("Loaded in {} Test pixels | {} Train pixels\n".format(len(test_pixels), len(train_pixels)))

# Loading in SIFT features:
SIFT_features = [[],[],[],[],[],[],[]]
new_path = os.path.join(path, "NPY files for BoVW")
for idx, i in enumerate([10, 100, 1000, 10000, 20000, 50000, 100000]):
    path = os.path.join(new_path, "bovw files for {} Words".format(i))
    test = np.load(os.path.join(path, "BoW_Test.npy"), allow_pickle=True)
    train = np.load(os.path.join(path, "BoW_Train.npy"), allow_pickle=True)
    SIFT_features[idx].append(test)
    SIFT_features[idx].append(train)
    print("Loaded in {} Test {}-dim SIFT features | {} Train {}-dim SIFT features".format(len(test_images), i, len(train_images), i))
    
print("\nLoading Complete!\n")
print("File names: test_images, train_images, test_names, train_names, test_pixels, train_pixels, SIFT_features")

Loaded in 55 Test images | 567 Train images

Loaded in 55 Test names | 567 Train names

Loaded in 55 Test pixels | 567 Train pixels

Loaded in 55 Test 10-dim SIFT features | 567 Train 10-dim SIFT features
Loaded in 55 Test 100-dim SIFT features | 567 Train 100-dim SIFT features
Loaded in 55 Test 1000-dim SIFT features | 567 Train 1000-dim SIFT features
Loaded in 55 Test 10000-dim SIFT features | 567 Train 10000-dim SIFT features
Loaded in 55 Test 20000-dim SIFT features | 567 Train 20000-dim SIFT features
Loaded in 55 Test 50000-dim SIFT features | 567 Train 50000-dim SIFT features
Loaded in 55 Test 100000-dim SIFT features | 567 Train 100000-dim SIFT features

Loading Complete!

File names: test_images, train_images, test_names, train_names, test_pixels, train_pixels, SIFT_features


In [17]:
SIFT_features[0][0]

array([[0.07355445, 0.07490411, 0.06076237, ..., 0.10873266, 0.05436633,
        0.08634653],
       [0.02450643, 0.13725855, 0.07351928, ..., 0.04411157, 0.08087121,
        0.07351928],
       [0.0231921 , 0.12821013, 0.05963683, ..., 0.07288946, 0.0927684 ,
        0.0695763 ],
       ...,
       [0.08292598, 0.07389165, 0.09121858, ..., 0.07463338, 0.10780378,
        0.08292598],
       [0.05177144, 0.07531628, 0.08135512, ..., 0.07026124, 0.02218776,
        0.09244899],
       [0.07477707, 0.05178143, 0.07178599, ..., 0.06281274, 0.08973248,
        0.09272357]])

## Metrics computation

In [23]:
def image_retrieval_k(train_data, test_data, train_names, test_names, train_images_as_array, test_images_as_array, k=20, view_option=0, image_size=(32,32), border_size=20):
    avg_precisions = []
    avg_recalls = []
    precisionsatk = []
    count = 0
    
    for idx, query in enumerate(test_data):
        
        all_precisions = []
        all_recalls = []
        precisions = []
        recalls = []

        # Finding the euclidean distance from the query image and sorting them into index
        query = query.reshape((1, -1))
        D = euclidean_distances(train_data, query).squeeze()
        index = np.argsort(D)
        
        # Finding the index of the last correct image in the sorted index to iter to
        last_correct_image_idx = 0
        for i in range(len(index)):
            if train_names[index[i]] == test_names[idx]:
                last_correct_image_idx = i
        
        # make sure we iter to k (for precision@k) if all correct images are found before k
        if k > last_correct_image_idx:
            last_correct_image_idx = k+1
        
        # Itering through all images untill we get to k or last correct image to compute AP
        for kk in range(1, last_correct_image_idx+2):
            TP = 0
            FP = 0
            FN = 0
            
            # Finding the correct amount of images in the training set
            correct_count = 0
            for ind in index:
                if train_names[ind] == test_names[idx]:
                    correct_count += 1
            sized_index = index[:kk]
            
            # Find TP FP FN
            for ind in sized_index:
                if train_names[ind] == test_names[idx]:
                    TP += 1
                else:
                    FP += 1
            FN = correct_count - TP
            
            # If we want to view the images then we run this code, else its a waste of computational time
            if view_option == 1:
                # Creating image of k images (including query image at start)
                tmp = [query.reshape(image_size)]
                for ind in sized_index[:k]:
                    tmp.append(train_data[ind].reshape(image_size))
                output = np.array(tmp)*255
                output = output.transpose(1, 0, 2)
                output = output.reshape((image_size[0], -1))
                im_query = Image.fromarray(output)
            
            # If the last k image is a correct image we add precision to the list
            if train_names[sized_index[-1]] == test_names[idx]:
                try:
                    precisions.append(TP/(TP+FP))
                except ZeroDivisionError:
                    precisions.append(0)
                try:
                    recalls.append(TP/(TP+FN))
                except ZeroDivisionError:
                    recalls.append(0)

            # Adding all precisions and recalls to a seperate list
            try:
                all_precisions.append(TP/(TP+FP))
            except ZeroDivisionError:
                all_precisions.append(0)
            try:
                all_recalls.append(TP/(TP+FN))
            except ZeroDivisionError:
                all_recalls.append(ZeroDivisionError)
     
        # Solving AP, AR and precision@k
        avg_precisions.append(np.average(precisions))
        avg_recalls.append(np.average(all_recalls))
        precisionsatk.append(all_precisions[k-1])
        
        # Set a viewing option, if 1 we print out the following:
        if view_option == 1:
            display(im_query) 
            print("Label: {}".format(test_names[idx]))
            print("Average Precision for query {}: ".format(idx), avg_precisions[-1])
            print("Precision@k for query {}: ".format(idx), precisionsatk[-1])
            print("\n")
        elif view_option == 0:
            count += 1 
            print("Percentage Complete: {}".format(round((count/len(test_data))*100),2), end="\r")
        elif view_option == 2:
            # Allowing a view_option 2 -> for viewing top k images from non_pixel value inputs
            # creating an array of the top k similar images
            top_k_images = [test_images_as_array[idx]]
            for i in range(0,k):
                top_k_images.append(train_images_as_array[index[i]])

            fig, axes = plt.subplots(1, k+1, figsize=(200/k, 200/k))
            for i, (image, ax) in enumerate(zip(top_k_images, axes.ravel())):
                # convert image to RGB and add border:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                # resize image if border size greater than 10:
                if border_size >= 10:
                    image = cv2.resize(image, (250, 400), interpolation=cv2.INTER_CUBIC)
                if i == 0:
                    query_name = test_names[idx]
                    title = "Query: {}".format(query_name)
                    color = (0, 255, 0)
                    image = border(image, color, border_size)
                else:
                    title = train_names[sized_index[i-1]]
                    if train_names[sized_index[i-1]] == query_name:
                        color = (0, 255, 0)
                        image = border(image, color, border_size)
                    else:
                        color = (255, 0, 0)
                        image = border(image, color, border_size)
                # display all set options
                ax.imshow(image, cmap="gray")
                ax.set_title(title)
                ax.axis("off")
            plt.show()
            print("Label: {}".format(test_names[idx]))
            print("Average Precision for query {}: ".format(idx), avg_precisions[-1])
            print("Precision@k for query {}: ".format(idx), precisionsatk[-1])
            print("\n")
        elif view_option == 3:
            top_k_images = [test_images_as_array[idx]]
            for i in range(0,k):
                top_k_images.append(train_images_as_array[index[i]])

            fig, axes = plt.subplots(1, k+1, figsize=(200/k, 200/k))
            for i, (image, ax) in enumerate(zip(top_k_images, axes.ravel())):
                # convert image to RGB and add border:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                # resize image if border size greater than 10:
                if border_size >= 10:
                    image = cv2.resize(image, (250, 400), interpolation=cv2.INTER_CUBIC)
                if i == 0:
                    query_name = test_names[idx]
                    title = "Query: {}".format(query_name)
                else:
                    title = train_names[sized_index[i-1]]
                    if train_names[sized_index[i-1]] == query_name:
                        color = (0, 255, 0)
                        image = border(image, color, border_size)
                    else:
                        color = (255, 0, 0)
                        image = border(image, color, border_size)
                # display all set options
                ax.imshow(image, cmap="gray")
                ax.set_title(title)
                ax.axis("off")
            plt.show()
    return avg_precisions, avg_recalls, precisionsatk

## Save metrics data to csv

In [ ]:
def save_data_to_csv(_precisionsatk, _AP, _k, _dataset_name):
    data = {'Precision@k': _precisionsatk, 'Average Precision': _AP}
    df = pandas.DataFrame(data=data)
    pandas.set_option("display.max_rows", 500, "display.max_columns", 4)
    df.to_csv('{}-metrics_k={}.csv'.format(_dataset_name, _k))

# Oxford 5k dataset func

In [ ]:
def Oxford5k_Image_Retrieval(images_path, gt_path, pixelorsift="sift", savedata=0, n_clusters=100, k=10, view_option=2):
    if pixelorsift == "pixel":
        # Load in data
        print("Loading Images...")
        test_gray_images, test_colour_images, test_names, test_imgs_path = load_test_data(images_path, gt_path)
        train_gray_images, train_colour_images, train_names = load_train_data(images_path, gt_path, test_imgs_path)
        
        # Compute pixel values
        print("\nComputing Pixel Values...")
        test_pixels, train_pixels = find_pixel_values(test_gray_images, train_gray_images)
        
        # Compute metrics
        print("\nComputing Metics...")
        AP, AR, precisionsatk = image_retrieval_k(train_pixels, test_pixels, train_names, test_names, train_colour_images, test_colour_images, k, view_option, border_size=20)

        # Display mAP
        mAP = np.average(AP)
        print("\nmAP =", mAP)
        
        # Save data
        if savedata == 1:
            save_data_to_csv(precisionsatk, AP, k, "Oxford5k_pixelvalues")
            print("\nData saved to csv")
        
    elif pixelorsift == "sift":
        # Load in data
        print("Loading Images...")
        test_gray_images, test_colour_images, test_names, test_imgs_path = load_test_data(images_path, gt_path)
        train_gray_images, train_colour_images, train_names = load_train_data(images_path, gt_path, test_imgs_path)
        
        # Copmuting bovw
        print("\nComputing test SIFT features...")
        test_kp, test_desc = SIFT(test_gray_images)
        print("\nComputing train SIFT features...")
        train_kp, train_desc = SIFT(train_gray_images)
        stacked_train_desc = stack_descriptors(train_desc)
        
        print("\nClustering Descriptors...")
        cluster_func = cluster(stacked_train_desc, n_clusters)
        
        print("\nComputing test BoVW...")
        test_bovw  = solve_BoW(test_desc, cluster_func, n_clusters)
        print("\nComputing train BoVW...")
        train_bovw = solve_BoW(train_desc, cluster_func, n_clusters)
        
        # Compute metrics
        print("\nComputing Metrics...")
        AP, AR, precisionsatk = image_retrieval_k(train_bovw, test_bovw, train_names, test_names, train_colour_images, test_colour_images, k, view_option, border_size=20)
        
        # Display mAP
        mAP = np.average(AP)
        print("\nmAP =", mAP)
        
        # Save data
        if savedata == 1:
            save_data_to_csv(precisionsatk, AP, k, "Oxford5k_BoVW_{}".format(n_clusters))
            print("\nData saved to csv")

    else:
        print("3rd input must be either: \"pixel\" or \"sift\"")

# MPEG7 dataset func

In [25]:
AP, AR, precisionsatk = image_retrieval_k(SIFT_features[0][1], SIFT_features[0][0], train_names, test_names, train_images_as_array=False, test_images_as_array=False)

In [27]:
np.mean(AP)

0.2846602906055916